# Import the required libraries

In [1]:
from SALib.sample import saltelli
from SALib.analyze import sobol
from apsimNGpy.senstivity.sensitivity import ConfigProblem, run_sensitivity

# Define APSIM model inputs as follows

In [2]:
params = {
       ".Simulations.Simulation.Field.Sow using a variable rule?Population": (2, 10),
       ".Simulations.Simulation.Field.Fertilise at sowing?Amount": (0, 300),
       ".Simulations.Simulation.Field.Maize.CultivarFolder.Dekalb_XL82?[Leaf].Photosynthesis.RUE.FixedValue": (
           1.2, 2.2),
   }

The base parameter path and the specific parameter name or sub-path are separated using one of the following delimiters: ?, ::, or @.

The values associated with each parameter define the lower and upper bounds of the sampling space used during sensitivity analysis.

# Initialize the problem as follows:

In [3]:
runner = ConfigProblem(
    base_model="Maize",
    params=params,
    outputs=["Yield", "Maize.AboveGround.N"],
)

SALib provides an object-oriented interface that simplifies the sensitivity analysis triad of sampling, evaluation, and analysis. In apsimNGpy, this workflow is further streamlined by wrapping all three steps into a single method, run_sensitivity().



In [ ]:
Si_sobol = run_sensitivity(
    runner,
    method="sobol",
    N=2 ** 6,  # ← base sample size should be power of 2
    engine = 'csharp',
    sample_options={
        "calc_second_order": False,
        "skip_values": 1024,
         "seed": 42,
    },
    analyze_options={
        "conf_level": 0.95,
        "num_resamples": 1000,
        "print_to_console": True,
        "calc_second_order": False,
    },
)

Processing 320 jobs wait..              0%  >> completed (elapsed=>00:00, eta=>?) 

The returned object is an instance of the SALib.util.problem.ProblemSpec class. When evaluated in the Python console, this object displays a summary of the problem definition and the computed sensitivity indices, as shown below.

# Converting outputs to pandas

In [ ]:
Si_sobol.to_df()

Key important attributes for samples, model results and analyses extraction available on the returned instance are shown below.

In [ ]:
print( Si_sobol.samples)
print( Si_sobol.results)
print( Si_sobol.analysis)

Attribute ``apsim_results`` allows you to extract the evaluation or APSIM simulated results

In [ ]:
Si_sobol.apsim_results

We can try another method known as Morris. The Morris method is typically used as a screening tool to identify influential parameters with relatively low computational cost. It is well suited for high-dimensional problems where the goal is to rank parameters rather than quantify precise sensitivities.

In [ ]:
Si_morris = run_sensitivity(
      runner,
      method="morris", n_cores=10,
      sample_options={
          'seed': 42,
          "num_levels": 6,
          "optimal_trajectories": 6,
      },
      analyze_options={
          'conf_level': 0.95,
          "num_resamples": 1000,
          "print_to_console": True,
          'seed': 42
      },
  )

The final sensitivity analysis method demonstrated in this tutorial is the FAST method. FAST is a variance-based approach that estimates the influence of each input parameter by systematically varying inputs across the parameter space and analyzing the resulting model response.

In [ ]:
si_fast = run_sensitivity(
    runner,
    method="fast",
    sample_options={
        "M": 2,

    },
    analyze_options={
        'conf_level': 0.95,
        "num_resamples": 1000,
        "print_to_console": True,
    },
)

Ideally, the analysis could begin with the Morris method to screen out less influential factors, particularly when a large number of inputs are under consideration. However, even without this initial screening, the results are consistent across methods.

In this case, both FAST and Sobol analyses indicate that yield is more sensitive to nitrogen fertilizer rate than to population density.

To extend the analysis beyond the Sobol, Morris, and FAST methods, follow the same workflow demonstrated in the first example and consult the SALib documentation for additional sensitivity analysis techniques and their usage.

- Sensitivity module was Added in version 1.0.0.
- version 1.1.0 provided improvements to the API by allowing specification of back-end simulation

# References

Iwanaga, T., Usher, W., & Herman, J. (2022). Toward SALib 2.0: Advancing the accessibility and interpretability of global sensitivity analyses. Socio-Environmental Systems Modelling, 4, 18155. doi:10.18174/sesmo.18155

Herman, J. and Usher, W. (2017) SALib: An open-source Python library for sensitivity analysis. Journal of Open Source Software, 2(9). doi:10.21105/joss.00097

